In [1]:
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Example usage
mechanism = 'Burke2012_s9r23.yaml'  # Example mechanism
gas_state = {
    'initial_temperature':  300,    # K
    'initial_pressure':     101325, # Pa
    'fuel_composition':     'H2:1',
    'oxidizer_composition': 'O2:0.21,N2:0.79',
    'equivalence_ratio':    1.0
}

import deepflame_node.one_d_flame_setup as odf

flame_speed, flame_thickness, final_flame = odf.calculate_laminar_flame_properties(mechanism, gas_state)
case_params = odf.update_case_parameters(mechanism, gas_state, flame_speed, flame_thickness)

odf.update_one_d_sample_config(case_params, gas_state)
odf.create_0_species_files(case_params)
odf.update_set_fields_dict(case_params)

In [ ]:
import subprocess

fields_str = f"T p {' '.join(case_params['unburnt_gas'].species_names)}"
command = f"reconstructPar -fields '({fields_str})'"
print(command)

# # Execute the command
# try:
#     result = subprocess.run(command, shell=True, check=True, text=True, capture_output=True)
#     print("Command executed successfully.")
#     print("Output:", result.stdout)
#     print("Error (if any):", result.stderr)
# except subprocess.CalledProcessError as e:
#     print("An error occurred while executing the command.")
#     print("Error code:", e.returncode)
#     print("Error output:", e.stderr)

In [ ]:
import os
import re
import json
import numpy as np
import cantera as ct

from pathlib import Path

from deepflame_node.utils import is_numeric_string

temp_gas = ct.Solution(mechanism)
sample_dims_list = ['T', 'p'] + temp_gas.species_names
print(sample_dims_list)

main_dir = Path('.')
time_dirs = [dir for dir in main_dir.iterdir() if dir.is_dir() and is_numeric_string(dir.name)]
sorted_time_dirs = sorted(time_dirs, key=lambda x: float(x.name))[1:]
print(len(sorted_time_dirs))

data_collector = []

for time_dir in sorted_time_dirs:
    time_dim_arrays = []
    
    for filename in sample_dims_list:
        file_path = time_dir / filename
        
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        sample_bool = False
        start_bracket_idx, end_bracket_idx = 0, 0
        for i, line in enumerate(lines):
            if "internalField" in line:
                sample_bool = True
            if "(" in line and sample_bool:
                start_bracket_idx = i + 1
            if ")" in line and sample_bool:
                end_bracket_idx = i
                break
        
        selected_lines = lines[start_bracket_idx:end_bracket_idx]
        
        dim_array = np.loadtxt(selected_lines).reshape(-1, 1)
        time_dim_arrays.append(dim_array)
    
    time_array = np.concatenate(time_dim_arrays, axis=1)
    data_collector.append(time_array)

data_array = np.concatenate(data_collector, axis=0)
print(data_array.shape)
print(data_array[0])

In [ ]:
collected_data = final_flame.collect_data(cols=['grid', 'T', 'Y', 'heat_release_rate'])
z = collected_data['grid']
T = collected_data['T']
Y = collected_data['Y']
heat_release_rate = collected_data['heat_release_rate']


fig, ax = plt.subplots(2, 1, figsize=(12, 6), dpi=600)

# Plot temperature profile
ax[0].plot(z, T, label='Temperature (K)')
ax1 = ax[0].twinx()
ax1.plot(z, heat_release_rate, label='Heat Release Rate [W/m³]', color='C1', linestyle='--')
ax[0].set_xlabel('Position (m)')
ax[0].set_ylabel('Temperature (K)')
ax[0].set_title('Flame Temperature Profile')
ax[0].grid()
ax[0].legend()

# Plot species mass fractions
for i, species in enumerate(final_flame.species_names):
    ax[1].plot(z, Y[:, i] / np.max(Y[:, i]), label=species)
ax[1].set_xlabel('Position (m)')
ax[1].set_ylabel('Normalized Mass Fraction')
ax[1].set_title('Flame Species Mass Fractions')
ax[1].grid()
ax[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
p = ct.one_atm  # pressure
tin_f = 300.0  # fuel inlet temperature
tin_o = 300.0  # oxidizer inlet temperature
mdot_o = 0.72  # kg/m^2/s
mdot_f = 0.24  # kg/m^2/s

comp_o = 'O2:0.21,N2:0.79'  # air composition
comp_f = 'H2:1'  # fuel composition

width = 0.02  # Distance between inlets is 2 cm

loglevel = 0  # amount of diagnostic output (0 to 5)

# Create the gas object used to evaluate all thermodynamic, kinetic, and
# transport properties.
gas = ct.Solution('Burke2012_s9r23.yaml')
gas.TP = gas.T, p

# Create an object representing the counterflow flame configuration,
# which consists of a fuel inlet on the left, the flow in the middle,
# and the oxidizer inlet on the right.
f = ct.CounterflowDiffusionFlame(gas, width=width)

# Set the state of the two inlets
f.fuel_inlet.mdot = mdot_f
f.fuel_inlet.X = comp_f
f.fuel_inlet.T = tin_f

f.oxidizer_inlet.mdot = mdot_o
f.oxidizer_inlet.X = comp_o
f.oxidizer_inlet.T = tin_o

# Set the boundary emissivities
f.boundary_emissivities = 0.0, 0.0
# Turn radiation off
f.radiation_enabled = False

f.set_refine_criteria(ratio=4, slope=0.2, curve=0.3, prune=0.04)

f.solve(loglevel, auto=True)



flame_result = f.to_solution_array()
collected_data = flame_result.collect_data(cols=['grid', 'T', 'Y', 'heat_release_rate'])

fig, ax = plt.subplots(dpi=600)
ax1 = ax.twinx()
ax.plot(
    collected_data['grid'], 
    (collected_data['T'] - min(collected_data['T'])) / (max(collected_data['T']) - min(collected_data['T'])), 
    linestyle='--'
)

for i, species in enumerate(gas.species_names):
    ax1.plot(
        collected_data['grid'], 
        collected_data['Y'][:, i],
        label=species,
    )
ax.set_title('Temperature of the flame')
# ax.set(ylim=(0,2500), xlim=(0.000, 0.020))
ax1.legend()

plt.show()
plt.close()